In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('')

In [3]:
db = client.get_database('production')

In [4]:
collection = db.get_collection('orders')

In [5]:
codes = db.get_collection('zipcodes')

In [6]:
import datetime as dt

In [7]:
now = dt.datetime.now()

In [8]:
current_month = now.month

In [9]:
records = collection.aggregate([
            {
                '$match': {
                    'address_id': {
                        '$exists': True
                    }, 
                    'createdOrderByShopify': None, 
                    'paid_status': 'PAGADO'
                }
            }, {
                '$lookup': {
                    'from': 'direcciones', 
                    'localField': 'address_id', 
                    'foreignField': '_id', 
                    'as': 'address'
                }
            }, {
                '$unwind': '$address'
            }, {
                '$match': {
                    'createdAt': {
                        '$gte': dt.datetime(2024, current_month, 1, 0, 0, 0, tzinfo=dt.timezone.utc), 
                        '$lt': dt.datetime.now()
                    }
                }
            }, {
                '$project': {
                    'cp': {
                        '$convert': {
                            'input': '$address.cp', 
                            'to': 'int', 
                            'onError': '', 
                            'onNull': ''
                        }
                    }, 
                    'kind_of_shop': 1
                }
            }, {
                '$group': {
                    '_id': {
                        'cp': '$cp', 
                        'kind_of_shop': '$kind_of_shop'
                    }, 
                    'sales': {
                        '$count': {}
                    }
                }
            },
            {
                '$project': {
                    'cp': '$_id.cp',
                    'kind_of_shop': '$_id.kind_of_shop',
                    'sales': 1,
                    '_id': 0
                }
            }
        ])

In [10]:
results = list(records)

In [13]:
import pandas as pd

In [54]:
from bson import ObjectId

In [14]:
df = pd.DataFrame(results)

In [15]:
df

,sales,cp,kind_of_shop
0,1,6720,BLIFE_ORDENES_MANUALES
1,1,64968,BLIFE_APP_MX
2,19,45138,BLIFE_APP_MX
3,1,58920,BLIFE_APP_MX
4,1,57920,BLIFE_APP_MX
...,...,...,...
3522,1,70150,BLIFE_APP_MX
3523,2,54945,BLIFE_APP_MX
3524,2,1010,BLIFE_APP_MX
3525,1,25198,BLIFE_APP_MX


In [16]:
df[df['sales'] > 1]

,sales,cp,kind_of_shop
2,19,45138,BLIFE_APP_MX
8,4,44200,BLIFE_APP_MX
9,3,64102,BLIFE_APP_MX
10,2,56334,BLIFE_APP_MX
13,4,64349,BLIFE_APP_MX
...,...,...,...
3514,3,56600,BLIFE_APP_MX
3515,2,99700,BLIFE_APP_MX
3520,3,21307,BLIFE_APP_MX
3523,2,54945,BLIFE_APP_MX


In [17]:
df['sales'].max()

np.int64(26)

In [26]:
zipcodes_records = codes.find()

In [27]:
df_zipcodes = pd.DataFrame(list(zipcodes_records))

In [31]:
df_zipcodes

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
0,66cf77f9dab1ab357d0e1b71,20000,Aguascalientes,Aguascalientes,Aguascalientes,1,[Aguascalientes Centro]
1,66cf77f9dab1ab357d0e1b72,20010,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas del Rio, Olivares Santana, Las Brisas..."
2,66cf77f9dab1ab357d0e1b73,20016,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Colinas de San Ignacio, La Fundición, Fundici..."
3,66cf77f9dab1ab357d0e1b74,20018,Aguascalientes,Aguascalientes,Aguascalientes,1,[Línea de Fuego]
4,66cf77f9dab1ab357d0e1b75,20020,Aguascalientes,Aguascalientes,Aguascalientes,1,"[Buenos Aires, Circunvalación Norte, Las Arbol..."
...,...,...,...,...,...,...,...
32205,66cf77f9dab1ab357d0e993e,99990,Moyahua de Estrada,,Zacatecas,32,[Alameda Juárez]
32206,66cf77f9dab1ab357d0e993f,99993,Moyahua de Estrada,,Zacatecas,32,[Cuxpala]
32207,66cf77f9dab1ab357d0e9940,99994,Moyahua de Estrada,,Zacatecas,32,[Vicente Guerrero]
32208,66cf77f9dab1ab357d0e9941,99997,Moyahua de Estrada,,Zacatecas,32,[Las Palmas]


In [35]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [92]:
def get_zipcode_id(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['_id'].tolist()
    print(res)
    if len(res) > 0:
        return res[0]
    return None

In [44]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id']

21120    66cf77f9dab1ab357d0e6df1
Name: _id, dtype: object

In [45]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'])

'21120    66cf77f9dab1ab357d0e6df1\nName: _id, dtype: object'

In [46]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [49]:
str(df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item())

'66cf77f9dab1ab357d0e6df1'

In [52]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()

[ObjectId('66cf77f9dab1ab357d0e6df1')]

In [83]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].tolist()[0]

ObjectId('66cf77f9dab1ab357d0e6df1')

In [63]:
df_zipcodes.loc[df_zipcodes['d_codigo'].eq(72120)]

,_id,d_codigo,D_mnpio,d_ciudad,d_estado,c_estado,d_asenta
21120,66cf77f9dab1ab357d0e6df1,72120,Puebla,Heroica Puebla de Zaragoza,Puebla,21,"[Los Arcos Santa Cruz, Ignacio Romero Vargas, ..."


In [78]:
df_zipcodes[df_zipcodes['d_codigo'].eq(72120)]['_id'].item()

ObjectId('66cf77f9dab1ab357d0e6df1')

In [93]:
df['zipcodes_id'] = df.apply(get_zipcode_id, axis=1)

[ObjectId('66cf77f9dab1ab357d0e305b')]
[ObjectId('66cf77f9dab1ab357d0e6314')]
[ObjectId('66cf77f9dab1ab357d0e4746')]
[ObjectId('66cf77f9dab1ab357d0e58d4')]
[ObjectId('66cf77f9dab1ab357d0e573a')]
[ObjectId('66cf77f9dab1ab357d0e8e13')]
[ObjectId('66cf77f9dab1ab357d0e78fd')]
[ObjectId('66cf77f9dab1ab357d0e7b0e')]
[ObjectId('66cf77f9dab1ab357d0e4667')]
[ObjectId('66cf77f9dab1ab357d0e6255')]
[ObjectId('66cf77f9dab1ab357d0e5631')]
[ObjectId('66cf77f9dab1ab357d0e1d8a')]
[ObjectId('66cf77f9dab1ab357d0e8849')]
[ObjectId('66cf77f9dab1ab357d0e62ab')]
[ObjectId('66cf77f9dab1ab357d0e2660')]
[ObjectId('66cf77f9dab1ab357d0e61ca')]
[ObjectId('66cf77f9dab1ab357d0e8391')]
[ObjectId('66cf77f9dab1ab357d0e51a4')]
[ObjectId('66cf77f9dab1ab357d0e30d5')]
[ObjectId('66cf77f9dab1ab357d0e61bc')]
[ObjectId('66cf77f9dab1ab357d0e5676')]
[ObjectId('66cf77f9dab1ab357d0e6eaf')]
[ObjectId('66cf77f9dab1ab357d0e6e80')]
[ObjectId('66cf77f9dab1ab357d0e6e74')]
[ObjectId('66cf77f9dab1ab357d0e29f6')]
[ObjectId('66cf77f9dab1ab

In [94]:
df

,sales,cp,kind_of_shop,zipcodes_id
0,1,6720,BLIFE_ORDENES_MANUALES,66cf77f9dab1ab357d0e305b
1,1,64968,BLIFE_APP_MX,66cf77f9dab1ab357d0e6314
2,19,45138,BLIFE_APP_MX,66cf77f9dab1ab357d0e4746
3,1,58920,BLIFE_APP_MX,66cf77f9dab1ab357d0e58d4
4,1,57920,BLIFE_APP_MX,66cf77f9dab1ab357d0e573a
...,...,...,...,...
3522,1,70150,BLIFE_APP_MX,66cf77f9dab1ab357d0e6a3b
3523,2,54945,BLIFE_APP_MX,66cf77f9dab1ab357d0e543b
3524,2,1010,BLIFE_APP_MX,66cf77f9dab1ab357d0e2edb
3525,1,25198,BLIFE_APP_MX,66cf77f9dab1ab357d0e2151


In [99]:
def get_state(row):
    res = df_zipcodes[df_zipcodes['_id'].eq(row['zipcodes_id'])]['d_estado'].tolist()
    print(res)
    if len(res) > 0:
        return res[0]
    return None

In [100]:
df['state'] = df.apply(get_state, axis=1)

['Ciudad de México']
['Nuevo León']
['Jalisco']
['Michoacán de Ocampo']
['México']
['Veracruz de Ignacio de la Llave']
['San Luis Potosí']
['Sinaloa']
['Jalisco']
['Nuevo León']
['México']
['Baja California']
['Tamaulipas']
['Nuevo León']
['Chiapas']
['Nayarit']
['Tabasco']
['México']
['Ciudad de México']
['Nayarit']
['México']
['Puebla']
['Puebla']
['Puebla']
['Chiapas']
['Hidalgo']
['Zacatecas']
['Jalisco']
['Sinaloa']
[]
['Quintana Roo']
['Veracruz de Ignacio de la Llave']
['Morelos']
['Puebla']
['Hidalgo']
['Baja California']
['Jalisco']
['México']
['Guanajuato']
['Nuevo León']
['Veracruz de Ignacio de la Llave']
['Colima']
['México']
['Coahuila de Zaragoza']
['Jalisco']
['Chiapas']
['Ciudad de México']
['Morelos']
['Morelos']
['Jalisco']
['Morelos']
['Ciudad de México']
['Guanajuato']
['Nuevo León']
['Guerrero']
['México']
['Querétaro']
['Oaxaca']
['Jalisco']
['Querétaro']
['Zacatecas']
['Nuevo León']
['Sinaloa']
['Ciudad de México']
['Chiapas']
['Querétaro']
['Morelos']
['Guerrer

In [101]:
df

,sales,cp,kind_of_shop,zipcodes_id,state
0,1,6720,BLIFE_ORDENES_MANUALES,66cf77f9dab1ab357d0e305b,Ciudad de México
1,1,64968,BLIFE_APP_MX,66cf77f9dab1ab357d0e6314,Nuevo León
2,19,45138,BLIFE_APP_MX,66cf77f9dab1ab357d0e4746,Jalisco
3,1,58920,BLIFE_APP_MX,66cf77f9dab1ab357d0e58d4,Michoacán de Ocampo
4,1,57920,BLIFE_APP_MX,66cf77f9dab1ab357d0e573a,México
...,...,...,...,...,...
3522,1,70150,BLIFE_APP_MX,66cf77f9dab1ab357d0e6a3b,Oaxaca
3523,2,54945,BLIFE_APP_MX,66cf77f9dab1ab357d0e543b,México
3524,2,1010,BLIFE_APP_MX,66cf77f9dab1ab357d0e2edb,Ciudad de México
3525,1,25198,BLIFE_APP_MX,66cf77f9dab1ab357d0e2151,Coahuila de Zaragoza


In [126]:
def get_state_name(row):
    res = df_zipcodes[df_zipcodes['d_codigo'].eq(row['cp'])]['d_estado'].tolist()
    print(res)
    if len(res) > 0:
        return res[0]
    return None

In [127]:
df['state_name'] = df.apply(get_state_name, axis=1)

['Ciudad de México']
['Nuevo León']
['Jalisco']
['Michoacán de Ocampo']
['México']
['Veracruz de Ignacio de la Llave']
['San Luis Potosí']
['Sinaloa']
['Jalisco']
['Nuevo León']
['México']
['Baja California']
['Tamaulipas']
['Nuevo León']
['Chiapas']
['Nayarit']
['Tabasco']
['México']
['Ciudad de México']
['Nayarit']
['México']
['Puebla']
['Puebla']
['Puebla']
['Chiapas']
['Hidalgo', 'Hidalgo']
['Zacatecas']
['Jalisco']
['Sinaloa']
[]
['Quintana Roo']
['Veracruz de Ignacio de la Llave']
['Morelos']
['Puebla']
['Hidalgo']
['Baja California']
['Jalisco']
['México']
['Guanajuato']
['Nuevo León']
['Veracruz de Ignacio de la Llave']
['Colima']
['México']
['Coahuila de Zaragoza']
['Jalisco']
['Chiapas']
['Ciudad de México']
['Morelos']
['Morelos']
['Jalisco']
['Morelos']
['Ciudad de México']
['Guanajuato']
['Nuevo León']
['Guerrero']
['México']
['Querétaro']
['Oaxaca']
['Jalisco']
['Querétaro']
['Zacatecas']
['Nuevo León']
['Sinaloa']
['Ciudad de México']
['Chiapas']
['Querétaro']
['Morelos'

In [128]:
df

,sales,cp,kind_of_shop,zipcodes_id,state,state_name
0,1,6720,BLIFE_ORDENES_MANUALES,66cf77f9dab1ab357d0e305b,Ciudad de México,Ciudad de México
1,1,64968,BLIFE_APP_MX,66cf77f9dab1ab357d0e6314,Nuevo León,Nuevo León
2,19,45138,BLIFE_APP_MX,66cf77f9dab1ab357d0e4746,Jalisco,Jalisco
3,1,58920,BLIFE_APP_MX,66cf77f9dab1ab357d0e58d4,Michoacán de Ocampo,Michoacán de Ocampo
4,1,57920,BLIFE_APP_MX,66cf77f9dab1ab357d0e573a,México,México
...,...,...,...,...,...,...
3522,1,70150,BLIFE_APP_MX,66cf77f9dab1ab357d0e6a3b,Oaxaca,Oaxaca
3523,2,54945,BLIFE_APP_MX,66cf77f9dab1ab357d0e543b,México,México
3524,2,1010,BLIFE_APP_MX,66cf77f9dab1ab357d0e2edb,Ciudad de México,Ciudad de México
3525,1,25198,BLIFE_APP_MX,66cf77f9dab1ab357d0e2151,Coahuila de Zaragoza,Coahuila de Zaragoza


In [115]:
grouped_by_state = df.groupby(df['state'])

In [116]:
grouped_by_state.count()

,sales,cp,kind_of_shop,zipcodes_id
state,,,,
Aguascalientes,62,62,62,62
Baja California,93,93,93,93
Baja California Sur,33,33,33,33
Campeche,17,17,17,17
Chiapas,55,55,55,55
Chihuahua,110,110,110,110
Ciudad de México,486,486,486,486
Coahuila de Zaragoza,78,78,78,78
Colima,31,31,31,31


In [117]:
grouped_by_state_kind_of_shop = df.groupby([df['state'], df['kind_of_shop']])

In [118]:
grouped_by_state_kind_of_shop.value_counts()

state           kind_of_shop            sales  cp     zipcodes_id             
Aguascalientes  BLIFE_APP_MX            1      20020  66cf77f9dab1ab357d0e1b75    1
                                               20030  66cf77f9dab1ab357d0e1b77    1
                                               20040  66cf77f9dab1ab357d0e1b78    1
                                               20050  66cf77f9dab1ab357d0e1b7a    1
                                               20059  66cf77f9dab1ab357d0e1b7b    1
                                                                                 ..
Zacatecas       BLIFE_APP_MX            4      98612  66cf77f9dab1ab357d0e972f    1
                BLIFE_ORDENES_MANUALES  1      98050  66cf77f9dab1ab357d0e9663    1
                MAYOREO                 1      98090  66cf77f9dab1ab357d0e967a    1
                                               98612  66cf77f9dab1ab357d0e972f    1
                                               99090  66cf77f9dab1ab357d0e97ea   

In [122]:
grouped_by_state_kind_of_shop.size()

state            kind_of_shop          
Aguascalientes   BLIFE_APP_MX              54
                 BLIFE_ORDENES_MANUALES     2
                 MAYOREO                    6
Baja California  BLIFE_APP_MX              82
                 BLIFE_ORDENES_MANUALES     4
                                           ..
Yucatán          BLIFE_APP_MX              29
                 MAYOREO                    4
Zacatecas        BLIFE_APP_MX              41
                 BLIFE_ORDENES_MANUALES     1
                 MAYOREO                    3
Length: 85, dtype: int64

In [123]:
grouped_by_state_kind_of_shop.size().to_frame().to_records()

rec.array([('Aguascalientes', 'BLIFE_APP_MX',  54),
           ('Aguascalientes', 'BLIFE_ORDENES_MANUALES',   2),
           ('Aguascalientes', 'MAYOREO',   6),
           ('Baja California', 'BLIFE_APP_MX',  82),
           ('Baja California', 'BLIFE_ORDENES_MANUALES',   4),
           ('Baja California', 'MAYOREO',   7),
           ('Baja California Sur', 'BLIFE_APP_MX',  30),
           ('Baja California Sur', 'BLIFE_ORDENES_MANUALES',   1),
           ('Baja California Sur', 'MAYOREO',   2),
           ('Campeche', 'BLIFE_APP_MX',  17),
           ('Chiapas', 'BLIFE_APP_MX',  47),
           ('Chiapas', 'BLIFE_ORDENES_MANUALES',   2),
           ('Chiapas', 'MAYOREO',   6),
           ('Chihuahua', 'BLIFE_APP_MX',  99),
           ('Chihuahua', 'BLIFE_ORDENES_MANUALES',   2),
           ('Chihuahua', 'MAYOREO',   9),
           ('Ciudad de México', 'BLIFE_APP_MX', 447),
           ('Ciudad de México', 'BLIFE_ORDENES_MANUALES',  25),
           ('Ciudad de México', 'MAYOREO',  14),


In [110]:
df['state'].value_counts()

state
Ciudad de México                   486
México                             442
Jalisco                            290
Nuevo León                         187
Puebla                             186
Guanajuato                         170
Veracruz de Ignacio de la Llave    160
Michoacán de Ocampo                136
Chihuahua                          110
Sonora                             104
Sinaloa                            100
Baja California                     93
Tamaulipas                          86
Querétaro                           80
Coahuila de Zaragoza                78
Morelos                             71
San Luis Potosí                     68
Oaxaca                              65
Aguascalientes                      62
Hidalgo                             58
Guerrero                            57
Chiapas                             55
Quintana Roo                        49
Tabasco                             48
Zacatecas                           45
Nayarit            

In [112]:
df['state'].value_counts().to_frame().to_records()

rec.array([('Ciudad de México', 486), ('México', 442), ('Jalisco', 290),
           ('Nuevo León', 187), ('Puebla', 186), ('Guanajuato', 170),
           ('Veracruz de Ignacio de la Llave', 160),
           ('Michoacán de Ocampo', 136), ('Chihuahua', 110),
           ('Sonora', 104), ('Sinaloa', 100), ('Baja California',  93),
           ('Tamaulipas',  86), ('Querétaro',  80),
           ('Coahuila de Zaragoza',  78), ('Morelos',  71),
           ('San Luis Potosí',  68), ('Oaxaca',  65),
           ('Aguascalientes',  62), ('Hidalgo',  58), ('Guerrero',  57),
           ('Chiapas',  55), ('Quintana Roo',  49), ('Tabasco',  48),
           ('Zacatecas',  45), ('Nayarit',  44), ('Durango',  35),
           ('Baja California Sur',  33), ('Yucatán',  33),
           ('Colima',  31), ('Tlaxcala',  24), ('Campeche',  17)],
          dtype=[('state', 'O'), ('count', '<i8')])

In [129]:
record_results = df['state'].value_counts().to_frame().to_records()

In [130]:
list(record_results)

[np.record(('Ciudad de México', 486), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('México', 442), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Jalisco', 290), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Nuevo León', 187), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Puebla', 186), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Guanajuato', 170), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Veracruz de Ignacio de la Llave', 160), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Michoacán de Ocampo', 136), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Chihuahua', 110), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sonora', 104), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Sinaloa', 100), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Baja California', 93), dtype=[('state', 'O'), ('count', '<i8')]),
 np.record(('Tamaulipas', 86), dtype=[('state', 'O'), ('count', '<i8')]),
 np.r

In [139]:
record_results = grouped_by_state_kind_of_shop.size().to_frame()

In [140]:
record_results

0
state           kind_of_shop              
Aguascalientes  BLIFE_APP_MX            54
                BLIFE_ORDENES_MANUALES   2
                MAYOREO                  6
Baja California BLIFE_APP_MX            82
                BLIFE_ORDENES_MANUALES   4
...                                     ..
Yucatán         BLIFE_APP_MX            29
                MAYOREO                  4
Zacatecas       BLIFE_APP_MX            41
                BLIFE_ORDENES_MANUALES   1
                MAYOREO                  3

[85 rows x 1 columns]

In [144]:
dict_result = record_results.to_dict('dict')

In [145]:
dict_result

{0: {('Aguascalientes', 'BLIFE_APP_MX'): 54,
  ('Aguascalientes', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Aguascalientes', 'MAYOREO'): 6,
  ('Baja California', 'BLIFE_APP_MX'): 82,
  ('Baja California', 'BLIFE_ORDENES_MANUALES'): 4,
  ('Baja California', 'MAYOREO'): 7,
  ('Baja California Sur', 'BLIFE_APP_MX'): 30,
  ('Baja California Sur', 'BLIFE_ORDENES_MANUALES'): 1,
  ('Baja California Sur', 'MAYOREO'): 2,
  ('Campeche', 'BLIFE_APP_MX'): 17,
  ('Chiapas', 'BLIFE_APP_MX'): 47,
  ('Chiapas', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chiapas', 'MAYOREO'): 6,
  ('Chihuahua', 'BLIFE_APP_MX'): 99,
  ('Chihuahua', 'BLIFE_ORDENES_MANUALES'): 2,
  ('Chihuahua', 'MAYOREO'): 9,
  ('Ciudad de México', 'BLIFE_APP_MX'): 447,
  ('Ciudad de México', 'BLIFE_ORDENES_MANUALES'): 25,
  ('Ciudad de México', 'MAYOREO'): 14,
  ('Coahuila de Zaragoza', 'BLIFE_APP_MX'): 75,
  ('Coahuila de Zaragoza', 'MAYOREO'): 3,
  ('Colima', 'BLIFE_APP_MX'): 28,
  ('Colima', 'MAYOREO'): 3,
  ('Durango', 'BLIFE_APP_MX'): 34,
  ('Duran

In [163]:
count_total = grouped_by_state_kind_of_shop['state'].count().reset_index(name='total')

In [164]:
count_total

,state,kind_of_shop,total
0,Aguascalientes,BLIFE_APP_MX,54
1,Aguascalientes,BLIFE_ORDENES_MANUALES,2
2,Aguascalientes,MAYOREO,6
3,Baja California,BLIFE_APP_MX,82
4,Baja California,BLIFE_ORDENES_MANUALES,4
...,...,...,...
80,Yucatán,BLIFE_APP_MX,29
81,Yucatán,MAYOREO,4
82,Zacatecas,BLIFE_APP_MX,41
83,Zacatecas,BLIFE_ORDENES_MANUALES,1


In [165]:
count_total.to_dict('records')

[{'state': 'Aguascalientes', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 54},
 {'state': 'Aguascalientes',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 2},
 {'state': 'Aguascalientes', 'kind_of_shop': 'MAYOREO', 'total': 6},
 {'state': 'Baja California', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 82},
 {'state': 'Baja California',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 4},
 {'state': 'Baja California', 'kind_of_shop': 'MAYOREO', 'total': 7},
 {'state': 'Baja California Sur', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 30},
 {'state': 'Baja California Sur',
  'kind_of_shop': 'BLIFE_ORDENES_MANUALES',
  'total': 1},
 {'state': 'Baja California Sur', 'kind_of_shop': 'MAYOREO', 'total': 2},
 {'state': 'Campeche', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 17},
 {'state': 'Chiapas', 'kind_of_shop': 'BLIFE_APP_MX', 'total': 47},
 {'state': 'Chiapas', 'kind_of_shop': 'BLIFE_ORDENES_MANUALES', 'total': 2},
 {'state': 'Chiapas', 'kind_of_shop': 'MAYOREO', 'total': 6},
 {'state':

In [95]:
df.to_records()

rec.array([(   0,  1, 6720, 'BLIFE_ORDENES_MANUALES', ObjectId('66cf77f9dab1ab357d0e305b')),
           (   1,  1, 64968, 'BLIFE_APP_MX', ObjectId('66cf77f9dab1ab357d0e6314')),
           (   2, 19, 45138, 'BLIFE_APP_MX', ObjectId('66cf77f9dab1ab357d0e4746')),
           ...,
           (3524,  2, 1010, 'BLIFE_APP_MX', ObjectId('66cf77f9dab1ab357d0e2edb')),
           (3525,  1, 25198, 'BLIFE_APP_MX', ObjectId('66cf77f9dab1ab357d0e2151')),
           (3526,  1, 57718, 'BLIFE_APP_MX', ObjectId('66cf77f9dab1ab357d0e5728'))],
          dtype=[('index', '<i8'), ('sales', '<i8'), ('cp', 'O'), ('kind_of_shop', 'O'), ('zipcodes_id', 'O')])